In [1]:
# | default_exp classes.DomoBootstrap

In [2]:
# | exporti
from dataclasses import dataclass, field
from fastcore.basics import patch_to
import httpx
import asyncio

import domolibrary.classes.DomoPage as dmpg

import domolibrary.utils.DictDot as util_dd
import domolibrary.client.DomoAuth as dmda
import domolibrary.client.DomoError as de
import domolibrary.routes.bootstrap as bootstrap_routes
import domolibrary.utils.chunk_execution as ce

In [3]:
# | export
@dataclass
class DomoBootstrap_Feature:
    id: int
    name: str
    label: str
    type: str
    purchased: bool
    enabled: bool

    @classmethod
    def _from_json_bootstrap(cls, json_obj: dict):
        dd = util_dd.DictDot(json_obj)

        bsf = cls(
            id=dd.id,
            name=dd.name,
            label=dd.label,
            type=dd.type,
            purchased=dd.purchased,
            enabled=dd.enabled,
        )
        return bsf

In [4]:
# | export
@dataclass
class DomoBootstrap:
    auth: dmda.DomoAuth = field(repr=False)
    bootstrap: dict = field(default=None)
    customer_id: str = None
    page_ls: list[dmpg.DomoPage] = field(default=None)
    feature_ls: list[DomoBootstrap_Feature] = field(default=None)

In [5]:
# | export
class Bootstrap_RetrievalError(de.DomoError):
    def __init__(self, status, response, domo_instance):
        super().__init__(status=status, response=response, domo_instance=domo_instance)


@patch_to(DomoBootstrap)
async def get_all(
    self: DomoBootstrap,
    auth: dmda.DomoFullAuth = None,
    debug_api: bool = False,
    return_raw: bool = False,
    debug_num_stacks_to_drop=2,
):
    auth = auth or self.auth

    res = await bootstrap_routes.get_bootstrap(
        auth=auth,
        debug_api=debug_api,
        debug_num_stacks_to_drop=debug_num_stacks_to_drop,
        parent_class=self.__class__.__name__,
    )

    if not res.is_success:
        raise Bootstrap_RetrievalError(
            status=res.status, response=res.response, domo_instance=auth.domo_instance
        )

    self.bootstrap = res.response

    if return_raw:
        return res

    return res.response

#### sample implementation of get_all


In [6]:
import os

full_auth = dmda.DomoFullAuth(
    domo_instance="domo-community",
    domo_password=os.environ["DOJO_PASSWORD"],
    domo_username=os.environ["DOMO_USERNAME"],
)

bs = DomoBootstrap(auth=full_auth)

res = await bs.get_all(return_raw=True, debug_api=False)

bs.bootstrap["data"].keys()

# res.traceback_details.function_trail

dict_keys(['landingPageId', 'favorites', 'pages', 'conf', 'analytics', 'brandKit', 'environmentId', 'features', 'authorities', 'customerHash', 'locale', 'isWhitelabeled', 'isCustomerWhitelabeled', 'isFullWhitelabeled', 'trialEndDate', 'trialContactPhone', 'whitelabeledProductName', 'userAnonymous', 'userIsProxied', 'domoappsDomain', 'customerTier', 'companyTimeZone', 'disableAutoRefresh', 'wasTrial', 'buzzSignup', 'authorizedAppDomains', 'isDomoAppDomainWhitelistEnabled', 'configProperties', 'domoPublicUrl', 'advancedColorControls', 'gettingStartedPageEnabled', 'landingEntities', 'isSignupSingleSolutionInstance', 'signupApp', 'cardPageMaxConcurrentRenders', 'version'])

In [16]:
# |exporti
@patch_to(DomoBootstrap)
async def get_customer_id(
    self: DomoBootstrap,
    auth: dmda.DomoFullAuth = None,
    debug_api: bool = False,
    debug_num_stacks_to_drop=3,
):
    await self.get_all(
        auth=auth or self.auth,
        debug_api=debug_api,
        debug_num_stacks_to_drop=debug_num_stacks_to_drop,
    )

    self.customer_id = self.bootstrap["currentUser"]["USER_GROUP"]
    return self.customer_id

#### sample implementation of get_customer_id


In [17]:
import os
import pandas as pd

full_auth = dmda.DomoFullAuth(
    domo_instance="domo-community",
    domo_password=os.environ["DOJO_PASSWORD"],
    domo_username=os.environ["DOMO_USERNAME"],
)

bs = DomoBootstrap(auth=full_auth)

await bs.get_customer_id(debug_api=False)

'mmmm-0012-0200'

In [9]:
# | exporti


@patch_to(DomoBootstrap)
async def get_pages(
    self: DomoBootstrap,
    auth: dmda.DomoFullAuth = None,
    return_raw: bool = False,
    debug_api: bool = False,
    debug_num_stacks_to_drop=2,
) -> list[dmpg.DomoPage]:
    auth = auth or self.auth

    res = await bootstrap_routes.get_bootstrap_pages(
        auth=auth,
        debug_api=debug_api,
        parent_class=self.__class__.__name__,
        debug_num_stacks_to_drop=debug_num_stacks_to_drop,
    )

    if return_raw:
        return res

    if not res.is_success:
        return None

    self.page_ls = await ce.gather_with_concurrency(
        n=60,
        *[
            dmpg.DomoPage._from_bootstrap(page_obj, auth=auth)
            for page_obj in res.response
        ]
    )

    return self.page_ls

#### sample implementation of get_pages


In [18]:
import os
import pandas as pd

full_auth = dmda.DomoFullAuth(
    domo_instance="domo-community",
    domo_password=os.environ["DOJO_PASSWORD"],
    domo_username=os.environ["DOMO_USERNAME"],
)

bs = DomoBootstrap(auth=full_auth)

page_ls = await bs.get_pages(return_raw=False, debug_api=False)

page_ls
# pd.DataFrame(page_ls[10:14]).drop(columns=["auth"])

[DomoPage(id=-100000, title='Overview', top_page_id=None, parent_page_id=None, is_locked=None, collections=[], owners=[], cards=[], custom_attributes={}, layout={}),
 DomoPage(id=-100003, title='Favorites', top_page_id=None, parent_page_id=None, is_locked=None, collections=[], owners=[], cards=[], custom_attributes={}, layout={}),
 DomoPage(id=-100002, title='Shared', top_page_id=None, parent_page_id=None, is_locked=None, collections=[], owners=[], cards=[], custom_attributes={}, layout={}),
 DomoPage(id=30507758, title='Welcome', top_page_id=None, parent_page_id=None, is_locked=None, collections=[], owners=[DomoGroup(id=1814479647, name='Admin Test', type='closed', description='', custom_attributes={}), DomoUser(id='587894148', display_name='Bryan Van Kampen', email_address='bryan@bvankampen.net', role_id=1, department=None, title=None, avatar_key=None, phone_number=None, web_landing_page=None, web_mobile_landing_page=None, employee_id=None, employee_number=None, hire_date=None, repor

In [11]:
# | exporti
@patch_to(DomoBootstrap)
async def get_features(
    self: DomoBootstrap,
    auth: dmda.DomoAuth = None,
    debug_api: bool = False,
    return_raw: bool = False,
    debug_num_stacks_to_drop=2,
    session: httpx.AsyncClient = None,
):
    auth = auth or self.auth

    res = await bootstrap_routes.get_bootstrap_features(
        auth=auth,
        session=session,
        debug_api=debug_api,
        return_raw=return_raw,
        debug_num_stacks_to_drop=debug_num_stacks_to_drop,
        parent_class=self.__class__.__name__,
    )

    if return_raw:
        return res

    feature_list = [
        DomoBootstrap_Feature._from_json_bootstrap(json_obj)
        for json_obj in res.response
    ]

    return feature_list

In [12]:
import os
import pandas as pd

full_auth = dmda.DomoFullAuth(
    domo_instance="domo-community",
    domo_password=os.environ["DOJO_PASSWORD"],
    domo_username=os.environ["DOMO_USERNAME"],
)

bs = DomoBootstrap(auth=full_auth)

feature_ls = await bs.get_features(return_raw=False, debug_api=False)

pd.DataFrame(feature_ls[10:14])

,id,name,label,type,purchased,enabled
0,3440,admin-pages,,PREMIUM,True,True
1,3441,admin-slideshows,,PREMIUM,True,True
2,3442,admin-access-rights,,PREMIUM,True,True
3,3443,admin-tool-downloads,,PREMIUM,True,True


In [13]:
# | exporti
@patch_to(DomoBootstrap)
async def is_group_ownership_beta(
    self, debug_api: bool = False, return_raw: bool = False
):
    domo_feature_ls = await self.get_features(
        return_raw=return_raw, debug_api=debug_api, debug_num_stacks_to_drop=3
    )

    if return_raw:
        return domo_feature_ls

    match_accounts_v2 = next(
        (
            domo_feature
            for domo_feature in domo_feature_ls
            if domo_feature.name == "accounts-v2"
        ),
        None,
    )

    return True if match_accounts_v2 else False

In [14]:
import os
import pandas as pd

full_auth = dmda.DomoFullAuth(
    domo_instance="domo-community",
    domo_password=os.environ["DOJO_PASSWORD"],
    domo_username=os.environ["DOMO_USERNAME"],
)

bs = DomoBootstrap(auth=full_auth)

await bs.is_group_ownership_beta(debug_api=False)

False

In [1]:
# | hide
import nbdev

nbdev.nbdev_export()
!nbqa black 50_DomoBootstrap.ipynb

All done! ✨ 🍰 ✨
1 file left unchanged.
